### 分类的评估方法

以西瓜问题为例，假定瓜农拉来一车西瓜，我们用训练好的模型对这些西瓜进行判别，显然，错误率衡量了有多少比例的瓜被判别错误。但若我们关心的是“挑出的西瓜有多少比例是好瓜”，或者“所有好瓜中有多少比例被挑了出来”，那么错误率显然就不够用了

#### 查准率、查全率和混淆矩阵
- 查准率 - 准确率 - precision
  - 预测结果为正例样本中真实为正例的比例
- 查全率 - 召回率 - recall
  - 真正为正例的样本中预测结果为正例的比例
- 查准率和查全率是一对矛盾的度量，一般来说：
  - 查准率高时，查全率往往偏低
  - 查全率高时，查准率往往偏低
- 混淆矩阵 - confusion matrix
  - 将样例根据其真实类别于预测类别的组合划分为以下4种情形：
    - 真正例 TP true positive
    - 假正例 FP false positive
    - 真反例 TN true negative
    - 假反例 FN false negative
  - TP+FP+TN+FN=样本总数
  - 二分类的混淆矩阵
![二分类的混淆矩阵](src/binary_class_confusion_matrix.png)
  - 查准率 P 定义为 $P=\frac{TP}{TP+FP}$
  - 查全率 R 定义为 $R=\frac{TP}{T{+FN}}$
#### F1-score
- F1是基于查准率和查全率的调和平均
- 反映了模型的稳健性  
$$F1 = \frac{2 * precision * recall}{precision + recall}=$$

#### API
~~~python
sklearn.metrics.classification_report
~~~
- 分类报告函数
- y_true: 真实目标值
- y_pred: 预测目标值
- labels: 指定类别对应数字
- target_names: 目标类别名称
- 返回: 每个类别的查准率和查全率

In [8]:
%run 17_logistic.ipynb
# 查看precision、recall和f1-score
from sklearn.metrics import classification_report
report = classification_report(y_test,y_pred)
print(report)

coef_ [[1.19777169 0.10877967 0.73209957 0.60323232 0.12122898 1.48162508
  0.75112762 0.79980762 0.82133788]]
intercept_ [-1.0749105]
err 0.0935672514619883
score 0.9766081871345029
              precision    recall  f1-score   support

           2       0.98      0.98      0.98       111
           4       0.97      0.97      0.97        60

    accuracy                           0.98       171
   macro avg       0.97      0.97      0.97       171
weighted avg       0.98      0.98      0.98       171



假设一种特殊情况，共100个样本，99个癌症，1个非癌症 - 样本不均衡的情况
如果我全都预测正例 - 不负责的模型
- 准确率：99%
- 召回率：99/99=100%
- 精确率：99%
- F1-score: 2*99%*100% / 100%+99% = 99.497%
- AUC: 0.5
  - TPR = 100%
  - FPR = 1 / 1 = 100%
在样本不均衡的情况下，这些指标不准确。
### ROC曲线和AUC指标
- ROC 全称受试者工作特征（Receiver Operating Characteristic）
  - ROC的纵轴是真正例率（True Positive Rate,TPR）,$TPR=\frac{TP}{TP+FN}$
    - 所有真实类别为1的样本中，预测类别为1的比例
  - ROC的横轴是假正例率（False Positive Rate,FPR）,$FPR=\frac{FP}{TN+FP}$
    - 所有真实类别为0的样本中，预测类别为1的比例
  
![roc](src/roc_curve.png)

- AUC (Area under the Curve of ROC)ROC曲线下方的面积大小
  - 因为是在1x1的方格里求面积，AUC必在0~1之间。
  - 假设阈值以上是阳性，以下是阴性；
  - 若随机抽取一个阳性样本和一个阴性样本，分类器正确判断阳性样本的值高于阴性样本之概率=AUC
  - 简单说：***AUC值越大的分类器，正确率越高***。
- 从AUC判断分类器（预测模型）优劣的标准：
  - AUC = 1，是完美分类器，采用这个预测模型时，存在至少一个阈值能得出完美预测。绝大多数预测的场合，不存在完美分类器。
  - 0.5 < AUC < 1，优于随机猜测。这个分类器（模型）妥善设置阈值的话，能有预测价值。
  - AUC = 0.5，跟随机猜测一样（例：丢铜板），模型没有预测价值。
  - AUC < 0.5，比随机猜测还差；但只要总是反预测而行，就优于随机猜测。
  - 最终AUC的范围在[0.5,1]之间
~~~python
sklearn.metrics.roc_auc_score
~~~
- 计算ROC曲线下面积，即AUC
- y_true: 每个样本的真实类别。必须为0(反例)，1(正例)标记
- y_score: 预测得分，可以是正类的估计概率、置信值或分类器方法的返回值
- 返回：AUC
#### 总结
- AUC 只适合评价二分类
- AUC 非常适合评价样本不均衡中的分类器性能

In [14]:
from sklearn.metrics import roc_auc_score
import numpy as np
y_true = np.where(y_test > 2.5 ,1,0) 
auc = roc_auc_score(y_true,y_pred)
auc

0.9743243243243243